In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm

from datetime import datetime

import numpy as np

import pandas as pd

In [2]:
def convert_values(value):
    if isinstance(value, str):
        value = value.replace("€", "").strip()
        if 'k' in value:
            return int(float(value.replace("k", "")) * 1_000)
        elif 'm' in value:
            return int(float(value.replace("m", "")) * 1_000_000)
        elif 'bn' in value:
            return int(float(value.replace("bn", "")) * 1_000_000_000)
    return value

In [4]:
c_options = webdriver.ChromeOptions()
c_options.add_argument("--start-maximized")
c_driver = webdriver.Chrome(options=c_options)

c_driver.get('https://www.transfermarkt.com/')


current_year = datetime.now().year
years = list(range(2004, int(current_year) + 1))
top_leagues = {
    'Premier-League': {'name':'GB', 'tier': 1},
    'Championship': {'name':'GB', 'tier': 2},
    'LaLiga':  {'name':'ES', 'tier': 1},
    'LaLiga2': {'name':'ES', 'tier': 2},
    'Bundesliga': {'name':'L', 'tier': 1},
    '2-Bundesliga': {'name':'L', 'tier': 2},
    'Serie-A': {'name':'IT', 'tier': 1},
    'Serie-B': {'name':'IT', 'tier': 2},
    'Ligue-1': {'name':'FR', 'tier': 1},
    'Ligue-2': {'name':'FR', 'tier': 2},
}

In [5]:
teams_list = []
for k, v in tqdm(top_leagues.items()):
    league_tier = v['tier']
    league_id = v['name'] + str(league_tier)
    for year in years:
        url = f'https://www.transfermarkt.com/{k.lower()}/startseite/wettbewerb/{league_id}/plus/?saison_id={year}'
        c_driver.get(url=url)
        time.sleep(15)
        teams_element = c_driver.find_elements(By.XPATH, '/html/body/div/main/div[1]/div[1]/div[2]/div[2]/div/table/tbody/tr')
        teams_results = c_driver.find_elements(By.XPATH, '/html/body/div/main/div[1]/div[2]/div[2]/div[2]/table/tbody/tr')
        for team in teams_element:

            team_name = team.find_element(By.XPATH, './td[2]/a[1]').get_attribute('title')
            team_pos = None

            try:
                for result in teams_results:
                    team_id = result.find_element(By.XPATH, './td[3]/a[1]')
                    if team_id.get_attribute('title') == team_name:
                        team_pos = result.find_element(By.XPATH, './td[1]').text
                        break
            except Exception as e:
                team_pos = None


            
            titles = team.find_elements(By.XPATH, './td[2]/a[position()>1]/img')
            has_won_titles = len(titles) > 0

            team_info = {
                'year': year,
                'country': v['name'],
                'league': k,
                'tier': league_tier,
                'team': team_name,
                'position': team_pos,
                'squad_depth': team.find_element(By.XPATH, './td[3]/a').text,
                'avg_age': team.find_element(By.XPATH, './td[4]').text,
                'foreigners': team.find_element(By.XPATH, './td[5]').text,
                'avg_market_value': convert_values(team.find_element(By.XPATH, './td[6]').text),
                'market_value': convert_values(team.find_element(By.XPATH, './td[7]/a').text),
                'has_relegated': np.nan,
                'has_promoted': np.nan,
                'has_won_titles': has_won_titles
            }
            teams_list.append(team_info)

100%|██████████| 10/10 [1:35:31<00:00, 573.18s/it]


In [6]:
df = pd.DataFrame(teams_list)
df

,year,country,league,tier,team,position,squad_depth,avg_age,foreigners,avg_market_value,market_value,has_relegated,has_promoted,has_won_titles
0,2004,GB,Premier-League,1,Chelsea FC,1,31,24.9,24,10690000,331480000,NaN,NaN,False
1,2004,GB,Premier-League,1,Manchester United,3,37,24.7,25,7930000,293230000,NaN,NaN,True
2,2004,GB,Premier-League,1,Arsenal FC,2,37,23.9,29,6680000,247000000,NaN,NaN,True
3,2004,GB,Premier-League,1,Liverpool FC,5,38,25.3,26,5850000,222130000,NaN,NaN,False
4,2004,GB,Premier-League,1,Tottenham Hotspur,9,36,25.2,21,3530000,127200000,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4258,2024,FR,Ligue-2,2,Red Star FC,None,28,26.7,10,371000,10400000,NaN,NaN,False
4259,2024,FR,Ligue-2,2,Stade Lavallois,None,24,28.3,11,427000,10250000,NaN,NaN,False
4260,2024,FR,Ligue-2,2,AC Ajaccio,None,26,27.0,11,390000,10150000,NaN,NaN,False
4261,2024,FR,Ligue-2,2,FC Annecy,None,22,25.9,6,357000,7850000,NaN,NaN,False


In [12]:
def modify_df(df: pd.DataFrame):
    df_copy = df.copy()
    # Inicializar las columnas con NaN
    df_copy['will_promote'] = np.nan
    df_copy['will_relegate'] = np.nan
    df_copy['has_promoted'] = np.nan
    df_copy['has_relegated'] = np.nan

    for idx, row in df_copy.iterrows():
        year, team, current_tier = row['year'], row['team'], row['tier']

        # Verificar año siguiente
        if year != 2024:
            next_year = df_copy[(df_copy['year'] == year + 1) & (df_copy['team'] == team)]

            if not next_year.empty:  # Si existe registro del equipo en el año siguiente
                next_tier = next_year.iloc[0]['tier']

                if next_tier < current_tier:  # Promoción
                    df_copy.at[idx, 'will_promote'] = True
                    df_copy.at[idx, 'will_relegate'] = False
                elif next_tier > current_tier:  # Descenso
                    df_copy.at[idx, 'will_promote'] = False
                    df_copy.at[idx, 'will_relegate'] = True
                else:  # Mismo nivel
                    df_copy.at[idx, 'will_promote'] = False
                    df_copy.at[idx, 'will_relegate'] = False
            else:  # El equipo no aparece en el año siguiente
                if current_tier == 2:  # Si estaba en tier 2 y desaparece, asumimos descenso
                    df_copy.at[idx, 'will_promote'] = False
                    df_copy.at[idx, 'will_relegate'] = True
                else:  # Para tier 1 u otros casos, dejamos como NaN
                    df_copy.at[idx, 'will_promote'] = np.nan
                    df_copy.at[idx, 'will_relegate'] = np.nan

        # Verificar año anterior
        if year != df_copy['year'].min():  # Evitar el primer año del conjunto de datos
            prev_year = df_copy[(df_copy['year'] == year - 1) & (df_copy['team'] == team)]

            if not prev_year.empty:  # Si existe registro del equipo en el año anterior
                prev_tier = prev_year.iloc[0]['tier']

                if prev_tier > current_tier:  # Promoción
                    df_copy.at[idx, 'has_promoted'] = True
                    df_copy.at[idx, 'has_relegated'] = False
                elif prev_tier < current_tier:  # Descenso
                    df_copy.at[idx, 'has_promoted'] = False
                    df_copy.at[idx, 'has_relegated'] = True
                else:  # Mismo nivel
                    df_copy.at[idx, 'has_promoted'] = False
                    df_copy.at[idx, 'has_relegated'] = False
            else:  # El equipo no aparece en el año anterior
                if current_tier == 2:  # Si aparece directamente en tier 2, asumimos promoción
                    df_copy.at[idx, 'has_promoted'] = True
                    df_copy.at[idx, 'has_relegated'] = False
                else:  # Para tier 1 u otros casos, dejamos como NaN
                    df_copy.at[idx, 'has_promoted'] = np.nan
                    df_copy.at[idx, 'has_relegated'] = np.nan
    return df_copy

In [18]:
df = modify_df(df=df)

C:\Users\Tore\AppData\Local\Temp\ipykernel_14132\2323533209.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_copy.at[idx, 'will_promote'] = False
C:\Users\Tore\AppData\Local\Temp\ipykernel_14132\2323533209.py:27: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_copy.at[idx, 'will_relegate'] = False
C:\Users\Tore\AppData\Local\Temp\ipykernel_14132\2323533209.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_copy.at[idx, 'has_promoted'] = Fal

In [20]:
df.to_csv('./1st_2nd_tiers_top_5_leagues.csv')

TEST

In [24]:
spain_df = df[df['country'] == 'ES']
spain_df = spain_df[spain_df['year'] != 2024]
spain_df = spain_df[spain_df['year'] != 2004]
spain_df

,year,country,league,tier,team,position,squad_depth,avg_age,foreigners,avg_market_value,market_value,has_relegated,has_promoted,has_won_titles,will_promote,will_relegate
944,2005,ES,LaLiga,1,FC Barcelona,1.0,34,25.4,16,9630000,327500000,False,False,True,False,False
945,2005,ES,LaLiga,1,Real Madrid,2.0,36,25.4,13,7820000,281600000,False,False,False,False,False
946,2005,ES,LaLiga,1,Valencia CF,3.0,34,27.3,14,6280000,213550000,False,False,False,False,False
947,2005,ES,LaLiga,1,Atlético de Madrid,10.0,31,24.2,8,4330000,134150000,False,False,False,False,False
948,2005,ES,LaLiga,1,Deportivo de La Coruña,8.0,34,27.5,8,3940000,133800000,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1779,2023,ES,LaLiga2,2,Racing Ferrol,10.0,29,28.6,2,641000,18600000,False,True,False,False,False
1780,2023,ES,LaLiga2,2,FC Cartagena,14.0,42,27.4,12,369000,15500000,False,False,False,False,False
1781,2023,ES,LaLiga2,2,CD Eldense,16.0,33,28.0,7,397000,13100000,False,True,False,False,False
1782,2023,ES,LaLiga2,2,SD Amorebieta,19.0,28,27.1,6,455000,12750000,False,True,False,False,True


In [25]:
spain_df.to_csv('./spain_df.csv')